In [57]:
import recordlinkage
from recordlinkage.datasets import load_febrl1
import pandas as pd

In [43]:
dfA = load_febrl1()

In [44]:
dfA.head(3)

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-223-org,NaN,waller,6,tullaroop street,willaroo,st james,4011,wa,19081209,6988048
rec-122-org,lachlan,berry,69,giblin street,killarney,bittern,4814,qld,19990219,7364009
rec-373-org,deakin,sondergeld,48,goldfinch circuit,kooltuo,canterbury,2776,vic,19600210,2635962


# Record pairs / Indexing
- One to One
- Blocking
    - here based on postcode
- SortedNeighbourhood

In [47]:
indexer = recordlinkage.Index()
indexer.block("postcode")
candidate_links = indexer.index(dfA)
len(candidate_links)

920

In [49]:
compare_cl = recordlinkage.Compare()
compare_cl.string(
    "given_name", "given_name", method="jarowinkler", threshold=0.85, label="given_name"
)
compare_cl.string(
    "surname", "surname", method="jarowinkler", threshold=0.85, label="surname"
)
compare_cl.exact("date_of_birth", "date_of_birth", label="date_of_birth")
compare_cl.exact("suburb", "suburb", label="suburb")
compare_cl.exact("state", "state", label="state")
compare_cl.string("address_1", "address_1", threshold=0.85, label="address_1")
features = compare_cl.compute(candidate_links, dfA)

In [50]:
features

given_name  surname  date_of_birth  suburb  state  \
rec_id_1      rec_id_2                                                          
rec-335-org   rec-223-org          0.0      0.0              0       0      0   
rec-223-dup-0 rec-223-org          0.0      1.0              1       1      1   
              rec-335-org          0.0      0.0              0       0      0   
rec-301-dup-0 rec-122-org          0.0      0.0              0       0      1   
rec-301-org   rec-122-org          0.0      0.0              0       0      1   
...                                ...      ...            ...     ...    ...   
rec-299-dup-0 rec-299-org          1.0      1.0              1       1      1   
rec-491-dup-0 rec-491-org          1.0      1.0              1       1      1   
rec-8-org     rec-8-dup-0          1.0      1.0              1       1      1   
rec-222-dup-0 rec-222-org          1.0      1.0              1       1      1   
rec-83-org    rec-83-dup-0         1.0      1.0              1       1      1   

                            address_1  
rec_id_1      rec_id_2                 
rec-335-org   rec-223-org         0.0  
rec-223-dup-0 rec-223-org         1.0  
              rec-335-org         0.0  
rec-301-dup-0 rec-122-org         0.0  
rec-301-org   rec-122-org         0.0  
...                               ...  
rec-299-dup-0 rec-299-org         1.0  
rec-491-dup-0 rec-491-org         1.0  
rec-8-org     rec-8-dup-0         1.0  
rec-222-dup-0 rec-222-org         1.0  
rec-83-org    rec-83-dup-0        1.0  

[920 rows x 6 columns]

In [51]:
features.describe()

,given_name,surname,date_of_birth,suburb,state,address_1
count,920.000000,920.000000,920.000000,920.000000,920.000000,920.000000
mean,0.341304,0.370652,0.402174,0.318478,0.544565,0.396739
std,0.474405,0.483242,0.490603,0.466140,0.498281,0.489487
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [52]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

6.0    136
5.0    169
4.0     80
3.0     28
2.0      7
1.0    105
0.0    395
Name: count, dtype: int64

In [53]:
matches = features[features.sum(axis=1) > 3]
matches

,,given_name,surname,date_of_birth,suburb,state,address_1
rec_id_1,rec_id_2,,,,,,
rec-223-dup-0,rec-223-org,0.0,1.0,1,1,1,1.0
rec-10-org,rec-10-dup-0,1.0,1.0,1,1,1,1.0
rec-6-org,rec-6-dup-0,0.0,1.0,1,0,1,1.0
rec-190-org,rec-190-dup-0,1.0,1.0,0,1,1,1.0
rec-294-dup-0,rec-294-org,1.0,1.0,1,1,1,1.0
...,...,...,...,...,...,...,...
rec-299-dup-0,rec-299-org,1.0,1.0,1,1,1,1.0
rec-491-dup-0,rec-491-org,1.0,1.0,1,1,1,1.0
rec-8-org,rec-8-dup-0,1.0,1.0,1,1,1,1.0


In [54]:
print(len(matches))

385


In [58]:
duplicate_records = []
for i in matches.index:
    duplicate_records.append(dfA.loc[i[0]])
    duplicate_records.append(dfA.loc[i[1]])
duplicate_df = pd.DataFrame(duplicate_records)
duplicate_df

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec-223-dup-0,jamilla,wallner,6,tullaroop street,willaroo,st james,4011,wa,19081209,6988048
rec-223-org,NaN,waller,6,tullaroop street,willaroo,st james,4011,wa,19081209,6988048
rec-10-org,kayla,harrington,38,maltby circuit,coaling,coolaroo,3465,nsw,19150612,9004242
rec-10-dup-0,kayla,harrington,NaN,maltby circuit,coaling,coolaroo,3465,nsw,19150612,9004242
rec-6-org,NaN,trevorrow,16,dumas street,2/98-latchford barracks,mount ommaney,2281,wa,19530313,4811642
...,...,...,...,...,...,...,...,...,...,...
rec-8-dup-0,james,jeslser,53,howe crescent,nuffield v illage,st clair,4215,vic,19120111,5961736
rec-222-dup-0,emiiyl,coleman,108,chewings street,berkeleyv lge,wellington point,2550,nsw,19421221,7206933
rec-222-org,emiily,coleman,108,chewings street,berkeley vlge,wellington point,2550,nsw,19421221,7206933
rec-83-org,mitchell,berry,1,braine street,gowrie,smithfield,3188,nsw,19400503,2887622
